In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta 
import os

In [3]:
service = pd.read_csv("../Data/PAAC_Service_Table.csv")
service.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,route_id,route_no,route_name,trip_id,trip_stop_sequence,stop_id,stop_name,direction_id,arr_time,dep_time,wkdy_service,sat_service,sun_service
0,001-182,1,Freeport Road,1473-1582265,1,TP7076,LAYOVER PENN PARK TERMINAL - NO STOP,Outbound,6:15 AM,6:15 AM,Yes,No,No
1,001-182,1,Freeport Road,1473-1582265,2,P01620,EAST BUSWAY AT PENN STATION C,Outbound,6:16 AM,6:16 AM,Yes,No,No
2,001-182,1,Freeport Road,1473-1582265,3,P02670,LIBERTY AVE OPP SMITHFIELD ST,Outbound,6:18 AM,6:18 AM,Yes,No,No
3,001-182,1,Freeport Road,1473-1582265,4,P02310,LIBERTY AVE AT 7TH ST (CLARK BLDG),Outbound,6:20 AM,6:20 AM,Yes,No,No
4,001-182,1,Freeport Road,1473-1582265,5,P00920,7TH ST AT FT DUQUESNE BLVD,Outbound,6:21 AM,6:21 AM,Yes,No,No


In [3]:
# # First go through all trips to get two lists, one with all departure times from first stops
# # and second with (adjusted by recovery factor) arrival times for all last stops on trip.

# # Parameter for recovery time in minutes
# recovery_factor = 0.0

# departure_times = list()
# arrival_times = list()

# for i in range(route1.shape[0]):
    
#     if i == 0:
#         departure_times.append(datetime.strptime(route1.dep_time[i], '%I:%M %p'))
        
#     elif i == (route1.shape[0]-1):
        
#         if pd.isna(route1.arr_time[i]):
#             j = i-1
#             while(pd.isna(route1.arr_time[j])):
#                 j = j-1
#             arrival_times.append(datetime.strptime(route1.arr_time[j], '%I:%M %p'))
#         else:
#             arrival_times.append(datetime.strptime(route1.arr_time[i], '%I:%M %p'))
        
#     elif route1.trip_stop_sequence[i] == 1:
        
#         # Some cases where we have na in columns
#         if pd.isna(route1.arr_time[i-1]):
#             j = i-1
#             while(pd.isna(route1.arr_time[j])):
#                 j = j-1
#             arrival_times.append(datetime.strptime(route1.arr_time[j], '%I:%M %p'))
#         else:
#             arrival_times.append(datetime.strptime(route1.arr_time[i-1], '%I:%M %p'))

#         departure_times.append(datetime.strptime(route1.dep_time[i], '%I:%M %p'))
        
# # Accouting for recovery factor, we adjust arrival times
# adjusted_arrivals = [arrival_times[i] + ((arrival_times[i] - departure_times[i]) * (recovery_factor)) \
#                      for i in range(len(arrival_times))]

In [4]:
# trips_df = pd.DataFrame(zip(departure_times, adjusted_arrivals), columns = ["Departure", "Arrival"])

# # Assume that any trip that departs between the hours of 12am and 3am occurs on the next day
# for i in range(1, trips_df.shape[0]):
    
#     if trips_df.Departure[i].hour <= 3:
#         trips_df.loc[i, "Departure"] = trips_df.loc[i, "Departure"] + timedelta(days=1)
#         trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)
#     elif trips_df.Arrival[i].hour <= 3:
#         trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)

# trips_df = trips_df.sort_values("Departure").reset_index(drop=True)


In [5]:
# # Next we count how many busses overlap at any one time and record the maximums
# peak_vehicles = 0

# for i in range(trips_df.shape[0]):
    
#     if i != trips_df.shape[0]-1:
#         inservice_vehicles = 0
#         temp_arrival = trips_df.Arrival[i]
        
#         j=i

#         while trips_df.Departure[j] < temp_arrival:
#             inservice_vehicles += 1
#             j+=1
            
#             if j == trips_df.shape[0]-1: break

#         if inservice_vehicles > peak_vehicles:
#             peak_vehicles = inservice_vehicles

### Functionalize

In [4]:
def remove_complete_na_trips(schedule):
    '''
    '''
    
    trips = schedule.trip_id.unique()
    trips_to_keep = list()
    
    for trip in trips:
        temp = schedule[schedule.trip_id == trip]
        
        last_stop = max(temp.trip_stop_sequence)
        
        if not (pd.isna(temp.arr_time[temp.trip_stop_sequence == last_stop].iloc[0]) and
                pd.isna(temp.dep_time[temp.trip_stop_sequence == 1].iloc[0])):
            
            trips_to_keep.append(trip)
            
    mask = [t in trips_to_keep for t in schedule.trip_id]
    
    return schedule[mask]


In [7]:
def peak_vehicle(schedule, recovery_factor = 0.0):
    '''
    
    '''
    ## Remove trips that have no departure time & arrival time information
    schedule = remove_complete_na_trips(schedule)
    
    ## First order schedule by trips and stops
    schedule = schedule.sort_values(["trip_id", "trip_stop_sequence"]).reset_index(drop = True)
    
    ## Second go through all trips to get two lists. One with all departure times from first stops
    ## and second with (adjusted by recovery factor) arrival times for all last stops on trip.
    
    departure_times = list()
    arrival_times = list()

    for i in range(schedule.shape[0]):

        if i == 0:
            departure_times.append(datetime.strptime(schedule.dep_time[i], '%I:%M %p'))

        elif i == (schedule.shape[0]-1):

            if pd.isna(schedule.arr_time[i]):
                j = i-1
                while(pd.isna(schedule.arr_time[j])):
                    j = j-1
                arrival_times.append(datetime.strptime(schedule.arr_time[j], '%I:%M %p'))
            else:
                arrival_times.append(datetime.strptime(schedule.arr_time[i], '%I:%M %p'))

        elif schedule.trip_stop_sequence[i] == 1:

            # Some cases where we have na in columns
            if pd.isna(schedule.arr_time[i-1]):
                j = i-1
                while(pd.isna(schedule.arr_time[j])):
                    j = j-1
                arrival_times.append(datetime.strptime(schedule.arr_time[j], '%I:%M %p'))
            else:
                arrival_times.append(datetime.strptime(schedule.arr_time[i-1], '%I:%M %p'))

            departure_times.append(datetime.strptime(schedule.dep_time[i], '%I:%M %p'))

    # Accouting for recovery factor, we adjust arrival times
    adjusted_arrivals = [arrival_times[i] + ((arrival_times[i] - departure_times[i]) * (recovery_factor)) \
                         for i in range(len(arrival_times))]
    
    ## Third, create a dataframe that has all first and last stop times.
    trips_df = pd.DataFrame(zip(departure_times, adjusted_arrivals), columns = ["Departure", "Arrival"])

    # Assume that any trip that departs (or arrives) between the hours of 12am and 3am occurs on the next day
    for i in range(0, trips_df.shape[0]):
    
        if trips_df.Departure[i].hour <= 3:
            trips_df.loc[i, "Departure"] = trips_df.loc[i, "Departure"] + timedelta(days=1)
            trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)
        elif trips_df.Arrival[i].hour <= 3:
            trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)

    trips_df = trips_df.sort_values("Departure").reset_index(drop=True)
    
    ## Finally  count how many busses overlap at any one time and record the maximums
    peak_vehicles = 0

    for i in range(trips_df.shape[0]):

        if i != trips_df.shape[0]-1:
            inservice_vehicles = 0
            temp_arrival = trips_df.Arrival[i]

            j=i

            while trips_df.Departure[j] < temp_arrival:
                inservice_vehicles += 1
                j+=1

                if j == trips_df.shape[0]-1: break

            if inservice_vehicles > peak_vehicles:
                peak_vehicles = inservice_vehicles
                
    return peak_vehicles

In [11]:
# Second try pv
def peak_vehicle(schedule, recovery_factor = 0.0):
    '''
    Given a schedule, returns the peak number of vehicles required to fulfill that schedule.
    Recovery factor is the percent of the trip that must be spent recovering.
    '''
    
    ### return "-" for empty dataframes
    if schedule.shape[0] == 0:
        return "-"
    
    ## Remove trips that have no departure time & arrival time information
    schedule = remove_complete_na_trips(schedule)
    
    ## Remove any trips where the time difference between first and last stops is less than 5 minutes
    trips = list(schedule.trip_id.unique())
    
    ## Convert arr_times to date type
    schedule.arr_time = schedule.arr_time.apply(lambda x: x if pd.isna(x) else datetime.strptime(x, '%I:%M %p'))

    new_df = pd.DataFrame()

    for t in trips:
        temp = schedule[schedule.trip_id == t].sort_values("trip_stop_sequence").reset_index(drop = True)
        
        first_stop_time = temp['arr_time'].iloc[0]
        
        last_stop_time_ix = temp.shape[0]-1
        
        if pd.isna(schedule.arr_time.iloc[last_stop_time_ix]):
                last_stop_time_ix = last_stop_time_ix-1
                
                while(pd.isna(schedule.arr_time.iloc[last_stop_time_ix])):
                    last_stop_time_ix = last_stop_time_ix-1
                    
                last_stop_time = temp['arr_time'].iloc[last_stop_time_ix]
                
        else:
            last_stop_time = temp['arr_time'].iloc[last_stop_time_ix]
        
        time_thru = last_stop_time - first_stop_time

        if time_thru.seconds/60 <= 5:
            continue
        else:
            new_df = new_df.append(temp)
            
    schedule = new_df.reset_index(drop = True)
    
    ## First order schedule by trips and stops
    schedule = schedule.sort_values(["trip_id", "trip_stop_sequence"]).reset_index(drop = True)
    
    ## Second go through all trips to get two lists. One with all departure times from first stops
    ## and second with (adjusted by recovery factor) arrival times for all last stops on trip.
    
    departure_times = list()
    arrival_times = list()

    for i in range(schedule.shape[0]):

        if i == 0:
            departure_times.append(schedule.arr_time[i])

        elif i == (schedule.shape[0]-1):

            if pd.isna(schedule.arr_time[i]):
                j = i-1
                while(pd.isna(schedule.arr_time[j])):
                    j = j-1
                arrival_times.append(schedule.arr_time[j])
            else:
                arrival_times.append(schedule.arr_time[i])

        elif schedule.trip_stop_sequence[i] == 1:

            # Some cases where we have na in columns
            if pd.isna(schedule.arr_time[i-1]):
                j = i-1
                while(pd.isna(schedule.arr_time[j])):
                    j = j-1
                arrival_times.append(schedule.arr_time[j])
            else:
                arrival_times.append(schedule.arr_time[i-1])

            departure_times.append(schedule.arr_time[i])

    # Accouting for recovery factor, we adjust arrival times
    adjusted_arrivals = [arrival_times[i] + ((arrival_times[i] - departure_times[i]) * (recovery_factor)) \
                         for i in range(len(arrival_times))]
    
    ## Third, create a dataframe that has all first and last stop times.
    trips_df = pd.DataFrame(zip(departure_times, adjusted_arrivals), columns = ["Departure", "Arrival"])

    # Assume that any trip that departs (or arrives) between the hours of 12am and 3am occurs on the next day
    for i in range(0, trips_df.shape[0]):
    
        if trips_df.Departure[i].hour <= 3:
            trips_df.loc[i, "Departure"] = trips_df.loc[i, "Departure"] + timedelta(days=1)
            trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)
        elif trips_df.Arrival[i].hour <= 3:
            trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)

    trips_df = trips_df.sort_values("Departure").reset_index(drop=True)
    
    ## Finally  count how many busses overlap at any one time and record the maximums
    peak_vehicles = 0

    for i in range(trips_df.shape[0]):

        if i != trips_df.shape[0]-1:
            inservice_vehicles = 0
            temp_arrival = trips_df.Arrival[i]

            j=i

            while trips_df.Departure[j] < temp_arrival:
                inservice_vehicles += 1
                j+=1

                if j == trips_df.shape[0]-1: break

            if inservice_vehicles > peak_vehicles:
                peak_vehicles = inservice_vehicles
                
    return peak_vehicles

In [6]:
def O_n_M(dFrame, PV_c, VRH_c, VRM_c, annualization = 300):
    '''
    '''
    dFrame['O_and_M'] = dFrame['VRH'] * VRH_c * annualization + \
                        dFrame['VRM'] * VRM_c * annualization + \
                        dFrame['PV'] * PV_c
    
    dFrame['O_and_M_per_hr'] = dFrame.O_and_M / (dFrame.VRH * 300)
    
    return dFrame

## Implement

In [12]:
# Use function to get peak vehicle for all weekday routes
pv_df = pd.DataFrame(columns = ["Route", "PV"])

wd_service = service[(service.sat_service == "No") & (service.sun_service == "No")]
wd_service['route_no'] = wd_service.route_no.apply(lambda x: str(x))
routes = list(set(wd_service.route_no))

for route in routes:
    print(route)
    pv = peak_vehicle(wd_service[wd_service.route_no == route])
    pv_df = pv_df.append(pd.DataFrame(zip([route], [pv]), columns = ["Route", "PV"]))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


O12
P3
88
15
12
75
P7
O1
91
52L
P2
27
14
DQI
G31
BLLB
4
G3
40
65
71D
19L
11
48
P78
61A
51
G2
44
57
77
P10
24
71C
81
55
61B
Y46
83
41
16
22
MI
O5
P68
59
26
58
43
P67
93
13
86
29
71A
Y49
67
61D
P1
6
38
21
P16
64
P69
39
82
31
71
8
P17
51L
74
RED
61C
P13
P71
18
87
56
69
17
BLSV
Y45
53L
60
7
Y47
36
71B
79
P12
54
89
1
28X
Y1
20
P76
2


In [13]:
pv_df

,Route,PV
0,O12,9
0,P3,9
0,88,5
0,15,4
0,12,4
...,...,...
0,28X,6
0,Y1,8
0,20,4
0,P76,8


In [15]:
pv_df.to_csv("../Data/PV2.csv", index=False)

## Combine all to get O&M models

In [52]:
# Have to match formats
pv_df['Route'] = [str(i).lstrip('0') for i in pv_df.Route]

In [53]:
vrm = pd.read_csv("../PPT_East_Busway/Data/Route VRM & VRH.csv")
vrm['Route'] = [str(i).lstrip('0') for i in vrm.Route]

In [54]:
combined = pd.merge(vrm, pv_df)
O_and_M = O_n_M(dFrame=combined, PV_c=186102, VRH_c=99.71, VRM_c=1.56, annualization=300)
O_and_M

,Route,VRH,VRM,PV,O_and_M,O_and_M_per_hr
0,1,67.816667,1189.402,6,3.70185e+06,181.954
1,2,58.466667,1020.387,6,3.34307e+06,190.597
2,4,27.550000,351.923,3,1.54711e+06,187.188
3,6,33.316667,316.359,4,1.88907e+06,189.001
4,7,5.266667,67.482,1,375225,237.484
...,...,...,...,...,...,...
90,Y1,17.416667,274.618,8,2.13832e+06,409.248
91,Y45,10.583333,149.958,3,945066,297.658
92,Y46,68.733333,1248.416,7,3.94299e+06,191.222
93,Y47,40.483333,656.473,4,2.26262e+06,186.3


In [55]:
O_and_M.to_csv("../PPT_East_Busway/Data/O_and_M.csv", index = False)